In [1]:
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.0 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM,AutoTokenizer,TrainingArguments,Trainer
from peft import get_peft_model,LoraConfig,TaskType
from datasets import Dataset

In [3]:
data={
    "text":[
        "### Question:What is BigData?\n### Answer:Big Data is huge ammount of data",
        "### Question:How hadoop used in Bigdata?\n### Answer:Hapdoop used in bigdata for distributed stotage and parallel processing",
        "### Question:What is Spark\n### Spark is a framework in bigdata",
    ]
}
dataset=Dataset.from_dict(data)

In [4]:
model_name="tiiuae/falcon-rw-1b"
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [5]:
def tokenize(batch):
  tokens=tokenizer(
      batch['text'],
      padding="max_length",
      truncation=True,
      max_length=128
  )
  tokens['labels']=tokens['input_ids'].copy()
  return tokens
tokenized_dataset=dataset.map(tokenize,batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [6]:
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)

config.json: 0.00B [00:00, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [7]:
lora_config=LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['query_key_value'],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [8]:
model=get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [9]:
training_args=TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    learning_rate=2e-4,
    logging_steps=1,
    save_strategy="no",
    report_to="none"
)

In [10]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)
trainer.train()

/tmp/ipython-input-10-1041140387.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,5.384900
2,5.475100
3,4.418500
4,4.001500
5,3.447400
6,3.834300
7,2.982100
8,3.268500
9,2.495700


TrainOutput(global_step=9, training_loss=3.9231048425038657, metrics={'train_runtime': 9.4726, 'train_samples_per_second': 0.95, 'train_steps_per_second': 0.95, 'total_flos': 8364732973056.0, 'train_loss': 3.9231048425038657, 'epoch': 3.0})

In [11]:
model.save_pretrained("lora-finetuned")
tokenizer.save_pretrained("lora-finetuned")

('lora-finetuned/tokenizer_config.json',
 'lora-finetuned/special_tokens_map.json',
 'lora-finetuned/vocab.json',
 'lora-finetuned/merges.txt',
 'lora-finetuned/added_tokens.json',
 'lora-finetuned/tokenizer.json')